In [1]:
from MFEA_lib.tasks.Benchmark.IDPC_EDU import IDPC_EDU_benchmark
from MFEA_lib.model import MFEA_base, SA_MFEA, SMP_MFEA, MFEA1, MFEA_surrogate
from MFEA_lib.model.utils import *
from MFEA_lib.operators.Crossover import *
from MFEA_lib.operators.Mutation import *
from MFEA_lib.operators.Selection import *

In [3]:
tasks, IndClass = IDPC_EDU_benchmark.get_tasks(1)

Reading data...


100%|██████████| 24/24 [00:13<00:00,  1.72it/s]


In [3]:
from MFEA_lib.EA import Population
dim_uss = max([t.dim for t in tasks])
population = Population(
            IndClass,
            nb_inds_tasks = [100] * len(tasks), 
            dim = dim_uss,
            list_tasks= tasks,
            evaluate_initial_skillFactor = True
        )
sub_pop = population.ls_subPop[0].ls_inds


In [9]:
import torch
import torch.nn as nn
import numpy as np

class SurrogateModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, output_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.linear1(x)
        #x = self.linear2(x)
        return x

In [17]:
inputs

array([[40, 31, 15, ..., 28,  2, 28],
       [40, 36, 29, ..., 28, 16, 34],
       [21, 29, 33, ..., 24, 39, 35],
       ...,
       [34, 38, 20, ..., 30, 33, 25],
       [26,  4, 19, ..., 17, 32, 32],
       [12, 44, 14, ..., 29, 19, 26]])

In [18]:
inputs = np.array([inds.genes.flatten() for inds in sub_pop]) / 40
outputs = np.array([inds.eval(tasks[0]) for inds in sub_pop])

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(inputs, outputs, test_size=0.1)

In [24]:
input_dim = X_train.shape[1]
hidden_dim = input_dim // 2
output_dim = 1
learning_rate = 1e-6
num_epochs = 20
device = 'cpu'

surrogate_model = SurrogateModel(input_dim, hidden_dim, output_dim)
criteria = torch.nn.MSELoss()
optimizer = torch.optim.SGD(surrogate_model.parameters(), lr = learning_rate)
total_step = X_train.shape[0]

for epoch in range(num_epochs):
    for i in range(X_train.shape[0]):
        input = torch.Tensor(X_train[i])
        pred = surrogate_model(input)
        output = torch.Tensor([y_train[i]])
        
        loss = criteria(output, pred)
        print(pred, output, loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 5 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Step {i+1}/{total_step}, Loss {loss}')
    
    mse_avg = 0.0
    with torch.no_grad():
        for i in range(X_val.shape[0]):
            input = torch.Tensor(X_train[i])
            pred = surrogate_model(input)
            output = torch.Tensor([y_train[i]])
            loss = criteria(output, pred)
            mse_avg += loss
    
    print(f"Validate, {mse_avg/X_val.shape[0]}")

tensor([0.1048], grad_fn=<AddBackward0>) tensor([17.]) tensor(285.4476, grad_fn=<MseLossBackward0>)
tensor([0.3408], grad_fn=<AddBackward0>) tensor([13.]) tensor(160.2549, grad_fn=<MseLossBackward0>)
tensor([0.1571], grad_fn=<AddBackward0>) tensor([15.]) tensor(220.3123, grad_fn=<MseLossBackward0>)
tensor([0.1810], grad_fn=<AddBackward0>) tensor([14.]) tensor(190.9642, grad_fn=<MseLossBackward0>)
tensor([-0.0123], grad_fn=<AddBackward0>) tensor([15.]) tensor(225.3690, grad_fn=<MseLossBackward0>)
Epoch 1/20, Step 5/90, Loss 225.3689727783203
tensor([0.0034], grad_fn=<AddBackward0>) tensor([14.]) tensor(195.9047, grad_fn=<MseLossBackward0>)
tensor([-0.1061], grad_fn=<AddBackward0>) tensor([15.]) tensor(228.1944, grad_fn=<MseLossBackward0>)
tensor([-0.0524], grad_fn=<AddBackward0>) tensor([14.]) tensor(197.4698, grad_fn=<MseLossBackward0>)
tensor([-0.2268], grad_fn=<AddBackward0>) tensor([14.]) tensor(202.4006, grad_fn=<MseLossBackward0>)
tensor([-0.1694], grad_fn=<AddBackward0>) tensor([

In [2]:
baseModel = MFEA_surrogate.model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover= newSBX(nc = 2, gamma= 0.4, alpha= 6),
    crossover= IDPCEDU_Crossover(),
    mutation= IDPCEDU_Mutation(),
    selection= ElitismSelection()
)
solve = baseModel.fit(
    nb_generations = 5000, rmp = 0.3, nb_inds_each_task= 10, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

NameError: name 'IndClass' is not defined

In [ ]:
baseModel.render_history()

In [ ]:
baseModel = SA_MFEA.model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover= newSBX(nc = 2, gamma= 0.4, alpha= 6),
    crossover= IDPCEDU_Crossover(),
    mutation= IDPCEDU_Mutation(),
    selection= ElitismSelection()
)
solve = baseModel.fit(
    max_inds_each_task= list(np.zeros(shape= (len(tasks),), dtype= int) + 4),
    min_inds_each_task= list(np.zeros(shape= (len(tasks),), dtype= int) + 4),
    max_eval_each_task= list(np.zeros(shape= (len(tasks),), dtype= int) + 500 * 4 ),
    log_oneline= True,
    num_epochs_printed= 1,
    evaluate_initial_skillFactor= True,
    LSA= True
)

In [ ]:
smpModel = SMP_MFEA.model()

smpModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # tasks = GECCO20_benchmark_50tasks.get_items(ID = id),
    # crossover= SBX_Crossover(nc = 2),
    crossover= newSBX(nc = 2, gamma= 0.4, alpha = 6),
    mutation = GaussMutation(scale = 0.1),
    selection= ElitismSelection(random_percent= 0.),
    search = SHADE(len_mem= 30, p_ontop= 0.1)
)
smpModel.fit(
    nb_generations= 1000, nb_inds_each_task= 100, nb_inds_min= 30,
    lr = 0.15, p_const_intra= 0., p_mutate= 0.1, prob_search = 0.,
    nb_epochs_stop= 1000, swap_po= False,
    evaluate_initial_skillFactor= True
)